# Emojify!  

Have you ever wanted to make your text messages more expressive? Emojifier app will help you do that. 
So rather than writing:
>"Congratulations on the promotion! Let's get coffee and talk. Love you!"   

The emojifier can automatically turn this into:
>"Congratulations on the promotion! 👍 Let's get coffee and talk. ☕️ Love you! ❤️"

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.initializers import glorot_uniform
from emo_utils import *
import emoji
import matplotlib.pyplot as plt


### Overview of the model

<img src="images/emojifier-v2.png" style="width:700px;height:400px;"> <br>
<caption><center> Emojifier: A 2-layer LSTM sequence classifier. </center></caption>

#### Embedding Layer in Keras
The embedding dimension is 50.
<img src="images/embedding1.png" style="width:700px;height:250px;">
<caption><center> Embedding layer</center></caption>

#### Dataset EMOJISET


- X contains sentences (strings).
- Y contains an integer label between 0 and 4 corresponding to an emoji for each sentence.

<img src="images/data_set.png" style="width:700px;height:300px;">
<caption><center> EMOJISET - a classification problem with 5 classes. A few examples of sentences are given here. </center></caption>


In [11]:
X_train, Y_train = read_csv('data/train_emoji.csv')
X_test, Y_test = read_csv('data/tesss.csv')

In [12]:
maxLen = len(max(X_train, key=len).split())

#### Use pre-trained 50-dimensional GloVe embeddings.

In [13]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('data/glove.6B.50d.txt')

In [14]:
word = "cucumber"
idx = 289846
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(idx) + "th word in the vocabulary is", index_to_word[idx])

the index of cucumber in the vocabulary is 113317
the 289846th word in the vocabulary is potatos


In [15]:
def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()`. 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                                   # number of training examples
    X_indices = np.zeros((m,max_len))
    
    for i in range(m):                               
        sentence_words = X[i].lower().split()
        j = 0
        for w in sentence_words:
            X_indices[i, j] = word_to_index[w]
            j = j + 1
        
    
    return X_indices

In [16]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary 

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    
    # Initialize the embedding matrix as a numpy array of zeros.
    emb_matrix = np.zeros((vocab_len,emb_dim))
    
    
    for word, idx in word_to_index.items():
        emb_matrix[idx, :] = word_to_vec_map[word]

    
    # Define Keras embedding layer with the correct input and output sizes
    embedding_layer = Embedding(vocab_len,emb_dim,trainable = False)
    

   
    # Build the embedding layer, it is required before setting the weights of the embedding layer. 
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [17]:
def Emojify_V2(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    
    # Define sentence_indices as the input of the graph.
    sentence_indices = Input(shape = input_shape,dtype = 'int32')
    
    # Create the embedding layer pretrained with GloVe Vectors
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer
    embeddings = embedding_layer(sentence_indices)   
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    X = LSTM(units = 128,return_sequences = True)(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    X = LSTM(units = 128,return_sequences = False)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with 5 units
    X = Dense(5)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    model = Model(inputs = sentence_indices,outputs = X)
    
    return model

In [29]:
model = Emojify_V2((maxLen,), word_to_vec_map, word_to_index)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 10, 50)            20000050  
_________________________________________________________________
lstm_2 (LSTM)                (None, 10, 128)           91648     
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 645 

In [30]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [31]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 5)

In [32]:
model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle=True)

Epoch 1/50
5/5 [==============================] - 0s 21ms/step - loss: 1.5892 - accuracy: 0.2500
Epoch 2/50
5/5 [==============================] - 0s 17ms/step - loss: 1.5029 - accuracy: 0.3182
Epoch 3/50
5/5 [==============================] - 0s 18ms/step - loss: 1.4654 - accuracy: 0.3333
Epoch 4/50
5/5 [==============================] - 0s 19ms/step - loss: 1.3517 - accuracy: 0.4394
Epoch 5/50
5/5 [==============================] - 0s 19ms/step - loss: 1.2445 - accuracy: 0.5303
Epoch 6/50
5/5 [==============================] - 0s 18ms/step - loss: 1.1741 - accuracy: 0.5303
Epoch 7/50
5/5 [==============================] - 0s 20ms/step - loss: 1.0287 - accuracy: 0.5985
Epoch 8/50
5/5 [==============================] - 0s 19ms/step - loss: 0.9210 - accuracy: 0.6591
Epoch 9/50
5/5 [==============================] - 0s 19ms/step - loss: 0.7759 - accuracy: 0.7273
Epoch 10/50
5/5 [==============================] - 0s 18ms/step - loss: 0.6760 - accuracy: 0.7652
Epoch 11/50
5/5 [============

In [33]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = convert_to_one_hot(Y_test, C = 5)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print()
print("Test accuracy = ", acc)

2/2 [==============================] - 0s 5ms/step - loss: 0.3577 - accuracy: 0.8750

Test accuracy =  0.875


In [34]:
x_test = np.array(['lets play football'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices))))

lets play football ⚾


In [35]:
model.save("my_model")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: my_model/assets


In [2]:
model = tf.keras.models.load_model("my_model")


In [24]:
import tkinter as tk

from tkinter import ttk

win = tk.Tk()

win.title('EmojiFier')

''

In [25]:
def emojifyMessage():
    x_test = np.array([origMessage.get()])
    X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
    success = x_test[0] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices)))
    print(success)
    tk.Label(win, text = success).grid(row = 1, column = 1)

In [26]:
tk.Label(win, text="Your Message").grid(row=0)
tk.Label(win, text="Emojified message").grid(row=1)

origMessage = tk.Entry(win)


origMessage.grid(row=0, column=1)


tk.Button(win,text='Emojify', command=emojifyMessage).grid(row=3)


    
win.mainloop()